In [1]:
from typing import List

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table

import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff

from flask_cors import CORS

import pandas as pd

import os
import requests
import io

In [2]:
url='https://raw.githubusercontent.com/dkremlg/Booking-Curves/master/GUI_in.csv'
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=',')

df['Dprio']=df['Dprio'].astype('float')
df['NumPax']=df['NumPax'].astype('float')
df['Ideal_lower']=df['Ideal_lower'].astype('float')
df['Ideal_average_downweighted']=df['Ideal_average_downweighted'].astype('float')
df['Ideal_average_full']=df['Ideal_average_full'].astype('float')
df['Ideal_upper']=df['Ideal_upper'].astype('float')

dep_dates = df['DepDate'].unique()
dtime = df['dtime'].unique()

# Function for conditional coloring

In [3]:
COLORS = [
    {
        'background': '#FF0000',
        'text': 'black'
    },
    {
        'background': '#5DFC0A',
        'text': 'black'
    },
]

def cell_style(value1, value2, value3):

    # style = {}
    relative_value1 = value1 - value2
    relative_value2 = value1 - value3

    if relative_value1 < 0 or relative_value2 > 0:
            style = {
                'backgroundColor': COLORS[0]['background'],
                'color': COLORS[0]['text']
            }
    else:
            style = {
                'backgroundColor': COLORS[1]['background'],
                'color': COLORS[1]['text']
            }
    return style

# Function for table creation

In [4]:
def generate_table(dataframe, max_rows=50):
    rows = []
    for i in range(min(len(dataframe), max_rows)):
        row = []
        for col in dataframe.columns:
            if col == 'NumPax':
                value1 = dataframe.iloc[i][col]
                value2 = dataframe.iloc[i]['Ideal_lower']
                value3 = dataframe.iloc[i]['Ideal_upper']
                style = cell_style(value1, value2, value3)
                row.append(html.Td(value1, style=style))
            else:
                value = dataframe.iloc[i][col]
                row.append(html.Td(value))
        rows.append(html.Tr(row))

    return html.Table(
            # Header
            [html.Tr([html.Th(col) for col in dataframe.columns])] +

            # Body
            rows)

# Import external stylesheet

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Core code

In [ ]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    # Header
    html.H1(children='Booking curve monitoring system'),
    # Dropdown for departure dates
    html.Div([dcc.Dropdown(id = 'dropdown_depdates',
    options=[{
    'label': i,
    'value': i
    } for i in dep_dates],
    value='All Departure Dates')],
    style={'width': '48%', 'display': 'inline-block'}),
    # Dropdown for departure times
    html.Div([dcc.Dropdown(id = 'dropdown_dtime',
    options=[{
    'label': i,
    'value': i
    } for i in dtime],
    value='All Departure Times')],
    style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),
    html.Div(id='table-container'),
])

@app.callback(
    Output('table-container', 'children'), 
    [Input('dropdown_depdates', 'value'),
    Input('dropdown_dtime', 'value')])
def update_table(value_depdates, value_dtime):
    dff = df.loc[(df['DepDate'] == value_depdates)&(df['dtime'] == value_dtime),:] # update with your own logic
    return generate_table(dff)

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Apr/2019 11:30:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 11:30:25] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 11:30:25] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 11:30:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 11:30:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 11:30:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 11:30:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 11:30:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 11:30:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 11:30:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 11:30:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 11:30:52] "POST /_dash-update-component